<a href="https://colab.research.google.com/github/ravi-prakash1907/A-tracking-of-COVID-19/blob/master/label_comparision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Finding Best Way of Image Classification

In [13]:
## libraries
import pandas as pd
import numpy as np

import os
import re
import math
import random
import requests
import matplotlib.pyplot as plt
from statistics import mean

In [14]:
## data downloader
def downloadCSV(fileURL, saveAs='downloaded.csv'):
  req = requests.get(fileURL)
  fileURLContent = req.content
  csv_file = open(saveAs, 'wb')
  
  csv_file.write(fileURLContent)
  csv_file.close()

In [17]:
kmeans='https://raw.githubusercontent.com/ravi-prakash1907/Adaptive-Fingerprint-Enhancement/main/modelPredictions/groups/kMeansResults.csv?token=AJGAAOGDTHNBR3O455IOI7LA2BZQ4'
tagged='https://raw.githubusercontent.com/ravi-prakash1907/Adaptive-Fingerprint-Enhancement/main/foundDS/tagged.csv?token=AJGAAOCQDJZXFBJO4EZ75ULAZ5EUY'

In [18]:
downloadCSV(kmeans, saveAs='predictedClusters.csv')
downloadCSV(tagged, saveAs='tagged.csv')

---

In [20]:
predictedClusters = pd.read_csv('predictedClusters.csv')
predictedClusters.head(3)

,image_name,by_kMeans1,by_kMeans2,by_kMeans3,by_kMeans4,by_kMeans5,by_kMeans6,by_kMeans7,by_kMeans8,by_kMeans9,by_kMeans10
0,00000_00.bmp,2,1,2,3,3,2,3,1,3,1
1,00000_01.bmp,2,1,2,3,3,2,3,1,3,1
2,00000_02.bmp,2,1,2,1,3,2,3,1,3,2


In [19]:
## count of values in each cluster
for colName in predictedClusters.columns[1:]:
  print(predictedClusters.value_counts(colName), end="\n\n")

by_kMeans1
3    480
1    219
2    101
dtype: int64

by_kMeans2
2    472
3    167
1    161
dtype: int64

by_kMeans3
3    474
1    172
2    154
dtype: int64

by_kMeans4
2    478
3    209
1    113
dtype: int64

by_kMeans5
2    480
1    219
3    101
dtype: int64

by_kMeans6
3    399
2    320
1     81
dtype: int64

by_kMeans7
2    399
3    320
1     81
dtype: int64

by_kMeans8
3    339
1    307
2    154
dtype: int64

by_kMeans9
1    470
2    167
3    163
dtype: int64

by_kMeans10
3    375
1    305
2    120
dtype: int64



---

In [72]:
tagged = pd.read_csv('tagged.csv')
tagged.sort_values('imageName',inplace=True)
tagged.head()

,tagName,imageName,Ravi Prakash,Gyan,Shivom,Aleena Rodrigues,Juby jose,Dhanya,Prajwal Nimje,Sonu Tewatia,Keerthana S,dryCount,normalCount,wetCount
545,tag546,00000_00.bmp,dry,NaN,NaN,NaN,NaN,dry,NaN,dry,NaN,3,0,0
110,tag111,00000_01.bmp,dry,NaN,NaN,dry,normal,NaN,NaN,normal,dry,3,2,0
65,tag66,00000_02.bmp,dry,dry,dry,dry,NaN,NaN,NaN,dry,dry,6,0,0
135,tag136,00000_03.bmp,dry,NaN,NaN,dry,normal,NaN,NaN,dry,dry,4,1,0
546,tag547,00000_04.bmp,wet,NaN,NaN,NaN,NaN,dry,NaN,wet,NaN,1,0,2


In [73]:
tagged.drop(axis=1, columns=['tagName'], inplace=True)
tagged.head(3)

,imageName,Ravi Prakash,Gyan,Shivom,Aleena Rodrigues,Juby jose,Dhanya,Prajwal Nimje,Sonu Tewatia,Keerthana S,dryCount,normalCount,wetCount
545,00000_00.bmp,dry,NaN,NaN,NaN,NaN,dry,NaN,dry,NaN,3,0,0
110,00000_01.bmp,dry,NaN,NaN,dry,normal,NaN,NaN,normal,dry,3,2,0
65,00000_02.bmp,dry,dry,dry,dry,NaN,NaN,NaN,dry,dry,6,0,0


In [74]:
def getLabel(df, colList):
  df = df[colList]
  lbls = []
  for i in df.index:
    row = list(df.loc[i])
    #print("row = {}\ni = {}\n".format(row, i))
    maxIndex = row.index(max(row))
    col = colList[maxIndex][:-5] # 'Count' substring is ommitted
    lbls.append(col)
  return lbls

In [75]:
counterCol = ['dryCount', 'normalCount', 'wetCount']

labelsForAll = getLabel(tagged, counterCol)
tagged['aggrLabel'] = labelsForAll
tagged.head()

,imageName,Ravi Prakash,Gyan,Shivom,Aleena Rodrigues,Juby jose,Dhanya,Prajwal Nimje,Sonu Tewatia,Keerthana S,dryCount,normalCount,wetCount,aggrLabel
545,00000_00.bmp,dry,NaN,NaN,NaN,NaN,dry,NaN,dry,NaN,3,0,0,dry
110,00000_01.bmp,dry,NaN,NaN,dry,normal,NaN,NaN,normal,dry,3,2,0,dry
65,00000_02.bmp,dry,dry,dry,dry,NaN,NaN,NaN,dry,dry,6,0,0,dry
135,00000_03.bmp,dry,NaN,NaN,dry,normal,NaN,NaN,dry,dry,4,1,0,dry
546,00000_04.bmp,wet,NaN,NaN,NaN,NaN,dry,NaN,wet,NaN,1,0,2,wet


In [76]:
## removing columns with nans
tagged = tagged.dropna(1)
tagged.reset_index(inplace=True, drop=True)
tagged.drop(columns=counterCol, inplace=True)
tagged.head()

,imageName,Ravi Prakash,Sonu Tewatia,aggrLabel
0,00000_00.bmp,dry,dry,dry
1,00000_01.bmp,dry,normal,dry
2,00000_02.bmp,dry,dry,dry
3,00000_03.bmp,dry,dry,dry
4,00000_04.bmp,wet,wet,wet


In [77]:
## healthy labels with equal tags
def strictLabelReviser():
  cols = tagged.columns[1:]
  flag = False

  for i in tagged.index:
    assiged = tagged[cols[-1]][i]
    prev = False
    for col in cols[:-1]:
      if prev:
        if prev == tagged[col][i]:
          continue
        else:
          flag = False
          break
      
      if tagged[col][i] == assiged:
        flag = False
        break
      else:
        flag = True
    
    # updating 
    if flag:
      tagged[cols[-1]][i] = tagged[cols[0]][i]

In [78]:
## droping tagName col
tagged = tagged[['imageName', 'aggrLabel']]
tagged.head()

,imageName,aggrLabel
0,00000_00.bmp,dry
1,00000_01.bmp,dry
2,00000_02.bmp,dry
3,00000_03.bmp,dry
4,00000_04.bmp,wet


In [79]:
## counting the tags and compairing with knn results
tagged.value_counts('aggrLabel')

aggrLabel
dry       300
normal    252
wet       248
dtype: int64

In [80]:
#from sklearn.metrics import accuracy_score
total = 800
correct = 0

In [123]:
## fun to map tags with labels
def tagToLabel(tagMap):
  tag = np.array(tagged['aggrLabel'])
  for i in range(total):
    if tag[i] == 'dry':
      tag[i] = tagMap['dry']
    elif tag[i] == 'normal':
      tag[i] = tagMap['normal']
    elif tag[i] == 'wet':
      tag[i] = tagMap['wet']
  return tag

## accuracy
def getAcc(tagMap):
  tag = tagToLabel(tagMap)
  dfCols = predictedClusters.columns[1:]
  dfVals = ['']*len(dfCols)
  accDF = pd.DataFrame([dfVals], columns=dfCols)
  i = 0

  for col in predictedClusters.columns[1:]:
    correct = 0
    for i in range(total):
      if tag[i] == predictedClusters[col][i]:
        correct += 1
    accDF[col][0] = correct/total
    i += 1
  
  print("Accuracy: ")
  display(accDF)
  return

In [124]:
## mapping
tagMap = {'dry':1, 'normal':2, 'wet':3}
getAcc(tagMap)

Accuracy: 


,by_kMeans1,by_kMeans2,by_kMeans3,by_kMeans4,by_kMeans5,by_kMeans6,by_kMeans7,by_kMeans8,by_kMeans9,by_kMeans10
0,0.61,0.435,0.55875,0.41875,0.55875,0.24125,0.2825,0.6725,0.01,0.34875


In [127]:
## mapping
tagMap = {'dry':1, 'normal':3, 'wet':2}
getAcc(tagMap)

Accuracy: 


,by_kMeans1,by_kMeans2,by_kMeans3,by_kMeans4,by_kMeans5,by_kMeans6,by_kMeans7,by_kMeans8,by_kMeans9,by_kMeans10
0,0.55875,0.4725,0.49625,0.4625,0.61,0.2825,0.24125,0.65625,0.04625,0.465


In [129]:
## mapping
tagMap = {'dry':2, 'normal':1, 'wet':3}
getAcc(tagMap)

Accuracy: 


,by_kMeans1,by_kMeans2,by_kMeans3,by_kMeans4,by_kMeans5,by_kMeans6,by_kMeans7,by_kMeans8,by_kMeans9,by_kMeans10
0,0.41,0.04625,0.465,0.01,0.005,0.5875,0.0075,0.235,0.48375,0.33


In [130]:
## mapping
tagMap = {'dry':3, 'normal':1, 'wet':2}
getAcc(tagMap)

Accuracy: 


,by_kMeans1,by_kMeans2,by_kMeans3,by_kMeans4,by_kMeans5,by_kMeans6,by_kMeans7,by_kMeans8,by_kMeans9,by_kMeans10
0,0.005,0.5575,0.005,0.5525,0.41,0.0075,0.5875,0.13,0.435,0.1275


In [131]:
## mapping
tagMap = {'dry':2, 'normal':3, 'wet':1}
getAcc(tagMap)

Accuracy: 


,by_kMeans1,by_kMeans2,by_kMeans3,by_kMeans4,by_kMeans5,by_kMeans6,by_kMeans7,by_kMeans8,by_kMeans9,by_kMeans10
0,0.385,0.0075,0.43625,0.02875,0.03125,0.75125,0.13,0.1975,0.555,0.52375


In [132]:
## mapping
tagMap = {'dry':3, 'normal':2, 'wet':1}
getAcc(tagMap)

Accuracy: 


,by_kMeans1,by_kMeans2,by_kMeans3,by_kMeans4,by_kMeans5,by_kMeans6,by_kMeans7,by_kMeans8,by_kMeans9,by_kMeans10
0,0.03125,0.48125,0.03875,0.5275,0.385,0.13,0.75125,0.10875,0.47,0.205


**by_kMeans6**, **by_kMeans7** and **by_kMeans8**  

Hence, following are the best objective measures for the classification, as of now:  
1. `SqSum_RVRatio`
2. `variance` 
3. `avg_bdd`  
4. `avg_rtovratio`

Also, following values ca be assigned to the labels:  

1. wet : 1
2. normal : 2
3. dry : 3